# Tabelas

In [2]:
pip install tabula-py

Note: you may need to restart the kernel to use updated packages.


In [4]:
import tabula
import pandas as pd
from flask import jsonify
import uuid
import os
def generate_game_id():
    return str(uuid.uuid4())

In [6]:
import csv
import json

def save_json_data(data, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(data, file, ensure_ascii=False, indent=4)

def load_json_data(filepath):
    """
    Carrega os dados de um arquivo JSON.

    :param filepath: O caminho para o arquivo JSON.
    :return: Os dados carregados do arquivo JSON.
    """
    with open(filepath, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data

def create_json(data, json_file):
    with open(json_file, 'w', encoding='utf-8') as file:  # Certifique-se de especificar a codificação correta
        json.dump(data, file, indent=4, ensure_ascii=False)  # Especifica ensure_ascii=False para lidar com acentos
    
    print('criado JSON em ' + json_file)

def csv_to_json(csv_file, json_file):
    """
    Converte um arquivo CSV em um arquivo JSON.

    :param csv_file: Caminho para o arquivo CSV.
    :param json_file: Caminho para o arquivo JSON de saída.
    """
    # Lista para armazenar os dados do CSV
    data = []

    # Lendo o arquivo CSV e populando a lista de dados
    with open(csv_file, 'r', encoding='utf-8') as file:  # Certifique-se de especificar a codificação correta
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            data.append(row)

    # Escrevendo os dados em um arquivo JSON
    create_json(data, json_file)

In [5]:
def create_games_files(df_games, filepath):
    csv_file = filepath +'/games.csv'
    json_file = filepath +'/games.json'
    df_games.to_csv(csv_file, index=False)
    csv_to_json(csv_file, json_file)

# Criando tabela dos grupos

In [4]:
def create_zero_ranking_group(table_groups, filepath):
    teams_groupA = table_groups['Grupo A'].tolist()
    teams_groupB = table_groups['Grupo B'].tolist()
    # Criar DataFrames vazios com as colunas necessárias
    columns = ['Time', 'Pontos', 'Jogos', 'V', 'E', 'D', 'Gols_Pro', 'Gols_Contra', 'Saldo']
    df_groupA = pd.DataFrame(columns=columns)
    df_groupB = pd.DataFrame(columns=columns)

    # Adicionar nomes dos grupos
    df_groupA['Time'] = teams_groupA
    df_groupB['Time'] = teams_groupB

    # Inicializar outras colunas com zeros
    for col in columns[1:]:
        df_groupA[col] = 0
        df_groupB[col] = 0

    # Define o tipo de dados para as colunas
    dtypes = {'Pontos': int, 'Jogos': int, 'V': int, 'E': int, 'D': int, 'Gols_Pro': int, 'Gols_Contra': int, 'Saldo': int}
    df_groupA = df_groupA.astype(dtypes)
    df_groupB = df_groupB.astype(dtypes)
    df_groupA['ID'] = [generate_game_id() for _ in range(len(df_groupA))]
    df_groupB['ID'] = [generate_game_id() for _ in range(len(df_groupB))]
    
    df_groupA.to_csv(filepath + '/ranking_A.csv', index=False)
    df_groupB.to_csv(filepath  + '/ranking_B.csv', index=False)
    
    csv_to_json(filepath  + '/ranking_A.csv', filepath + '/ranking_A.json')
    csv_to_json(filepath + '/ranking_B.csv', filepath + '/ranking_B.json')

    return df_groupA, df_groupB

# Formatando as tabelas dos grupos

In [6]:
def format_table_group(tb_groups):
    df_groups = []
    # Para cada tabela de grupo
    for idx, tb_group in enumerate(tb_groups, start=1):
        # Criar um DataFrame a partir dos dados da tabela
        df_group_base = pd.DataFrame(tb_group)

        # Atualizando nome das colunas
        novos_nomes_colunas = ['Col.', 'Time', 'Pontos', 'Jogos', 'V', 'E', 'D', 'Gols_Pro', 'Gols_Contra', 'Saldo']
        df_group_base.columns = novos_nomes_colunas

        # Removendo a primeira coluna
        df_group = df_group_base.drop(columns=['Col.'])

        # Removendo as duas primeiras linhas (antiga coluna)
        df_group.drop([0, 1], inplace=True)

        # Preencher valores NaN com zero
        df_group.fillna(0, inplace=True)

        # Converter os dados das colunas para o tipo int
        colunas_para_converter = ['Pontos', 'Jogos', 'V', 'E', 'D', 'Gols_Pro', 'Gols_Contra', 'Saldo']
        df_group[colunas_para_converter] = df_group[colunas_para_converter].astype(int)

        # Imprimir o resultado
        # print(f"Grupo {chr(64+idx)}:")
        # print(df_group)
        # print("#########################################################################")
        df_groups.append(df_group)
        
    return df_groups
    


# Formatando tabela de jogos

In [30]:
def verify_empty_games(tables):
    for df in tables:
        # Verificar se 'EQUIPE Mandante' é NaN
        empty_rows = df[df['EQUIPE Mandante'].isna()]

        # Remover as linhas onde 'EQUIPE Mandante' é NaN
        if not empty_rows.empty:
            df.dropna(subset=['EQUIPE Mandante'], inplace=True)
    
    concatenated_df = pd.concat(tables, ignore_index=True)
    return concatenated_df

def generate_table_games(tables):
    tb_games = verify_empty_games(tables)
    # tb_games = pd.concat([tables[0], tables[1]], ignore_index=True)
    df_games = pd.DataFrame(tb_games)
    if df_games['DIA'].isna().all():
        df_games['DIA'] = ''
    else:
        # Limpar os espaços extras e formatar a coluna 'DIA'
        df_games['DIA'] = df_games['DIA'].str.replace(' ', '', regex=False)
        df_games['DIA'] = pd.to_datetime(df_games['DIA'] + '/' + pd.to_datetime('now', utc=True).strftime('%Y'), format='%d/%m/%Y', errors='coerce')

    # Limpar os espaços extras e corrigir a formatação dos placares
    df_games['PLACAR'] = df_games['PLACAR'].str.replace(' ', '', regex=True)
    df_games[['GOLS_MANDANTE', 'GOLS_VISITANTE']] = df_games['PLACAR'].str.split('X', expand=True)
    df_games['SIMULADOR'] = False
    df_games['ID'] = [generate_game_id() for _ in range(len(df_games))]
    df_games.rename(columns={'HORÁRIO': 'HORARIO'}, inplace=True)
    df_games.rename(columns={'EQUIPE Mandante': 'Mandante'}, inplace=True)
    df_games.rename(columns={'EQUIPE Visitante': 'Visitante'}, inplace=True)
    
    return df_games

# Jogos por time

In [8]:
def listar_jogos_por_time(df_games, team_surname):
  condition_home = df_games['EQUIPE Mandante'].str.contains(team_surname)
  condition_away = df_games['EQUIPE Visitante'].str.contains(team_surname)
  games_by_team = df_games[condition_home | condition_away]
  return games_by_team

def listar_confrontos(df_games, teamOne, teamTwo):
    # Filtrar os jogos onde a equipe é a mandante ou visitante
    condition_home = df_games['EQUIPE Mandante'].str.contains(teamOne)
    condition_away = df_games['EQUIPE Visitante'].str.contains(teamOne)

    # Combinar as condições usando o operador lógico OR (|)
    games_teamOne = df_games[condition_home | condition_away]

    # Filtrar os jogos onde a outra equipe é a visitante
    games_between_home = games_teamOne[games_teamOne['EQUIPE Mandante'].str.contains(teamTwo)]
    if games_between_home.empty == False:
      return games_between_home
    else:
      return games_teamOne[games_teamOne['EQUIPE Visitante'].str.contains(teamTwo)]

# Confronto direto

In [5]:
def update_confronto_direto(winner_team, loser_team, draw, confrontos_diretos):
   if draw:
      resultado = 'E'
   else:
      resultado = winner_team

  # Registrar o resultado no dicionário
   confrontos_diretos.setdefault(winner_team, {}).setdefault(loser_team, resultado)
   confrontos_diretos.setdefault(loser_team, {}).setdefault(winner_team, resultado)
   print(confrontos_diretos)
   return confrontos_diretos

def remover_confronto_direto(winner_team, loser_team, confrontos_diretos):
  # Registrar o resultado no dicionário
   confrontos_diretos.setdefault(winner_team, {}).setdefault(loser_team, '')
   confrontos_diretos.setdefault(loser_team, {}).setdefault(winner_team, '')
   return confrontos_diretos

def gerar_confronto_direto(df_games, filepath):
  confrontos_diretos = {}

  # Iterar sobre os jogos e registrar os resultados dos confrontos diretos
  for _, jogo in df_games.iterrows():
      equipe_mandante = jogo['Mandante']
      equipe_visitante = jogo['Visitante']
      placar_mandante = jogo['GOLS_MANDANTE']
      placar_visitante = jogo['GOLS_VISITANTE']

      resultado = ''

      if placar_mandante > placar_visitante:
          resultado = equipe_mandante
      elif placar_mandante < placar_visitante:
          resultado = equipe_visitante
      else:
          resultado = 'E'

      # Registrar o resultado no dicionário
      confrontos_diretos.setdefault(equipe_mandante, {}).setdefault(equipe_visitante, resultado)
      confrontos_diretos.setdefault(equipe_visitante, {}).setdefault(equipe_mandante, resultado)

  create_json(confrontos_diretos, filepath + '/confrontation.json')

  return confrontos_diretos

def confrontos_to_df(confrontos_diretos):
  # Criar um DataFrame a partir dos resultados dos confrontos diretos
  df_confrontos_diretos = pd.DataFrame(confrontos_diretos).T.fillna('').sort_index()
  df_confrontos_diretos.index.name = 'Time'
  return df_confrontos_diretos

# Filtro com classificados

In [10]:
def atualizar_classificacao(tb_groups, df_confrontos_diretos):
  for tb_group in tb_groups: 
    df_group = pd.DataFrame(tb_group['ranking'])
    linhas_pontos_iguais = df_group[df_group.duplicated(subset='Pontos', keep=False)]

    # Obter apenas os nomes das atléticas
    atleticas_pontos_iguais = linhas_pontos_iguais['Time'].tolist()
    if len(atleticas_pontos_iguais) == 2:
      team_ahead = df_confrontos_diretos.loc[atleticas_pontos_iguais[0], atleticas_pontos_iguais[1]]
      atleticas_pontos_iguais.remove(team_ahead)

      # Nome da atlética que você deseja trocar
      team_behind = atleticas_pontos_iguais[0]

      # Encontrar a posição da atlética no DataFrame
      position_ahead = df_group.index[df_group['Time'] == team_ahead].tolist()[0]
      position_behind = df_group.index[df_group['Time'] == team_behind].tolist()[0]

      # Trocar os valores entre as linhas diretamente
      df_group.loc[position_ahead], df_group.loc[position_behind] = df_group.loc[position_behind].copy(), df_group.loc[position_ahead].copy()

    print(f"Classificação grupo {tb_group['group']} atualizada com confrontos diretos:")
    df_group_sorted = df_group.sort_values(by=['Pontos', 'Saldo', 'Gols_Pro'], ascending=[False, False, False], inplace=True)
    print(df_group_sorted)
    # return df_group_sorted


In [23]:
def update_ranking(tb_groups, df_confrontos_diretos):
  df_groups = []
  idx = 1
  columns_to_convert = ['Pontos', 'Jogos', 'V', 'E', 'D', 'Gols_Pro', 'Gols_Contra', 'Saldo']
  for tb_group in tb_groups: 
    df_group = pd.DataFrame(tb_group)
    df_group[columns_to_convert] = df_group[columns_to_convert].astype(int)
    df_group.sort_values(by=['Pontos', 'Saldo', 'Gols_Pro'], ascending=[False, False, False], inplace=True)
    linhas_pontos_iguais = df_group[df_group.duplicated(subset='Pontos', keep=False)]

    # Obter apenas os nomes das atléticas
    atleticas_pontos_iguais = linhas_pontos_iguais['Time'].tolist()
    if len(atleticas_pontos_iguais) == 2:
      team_ahead = df_confrontos_diretos.loc[atleticas_pontos_iguais[0], atleticas_pontos_iguais[1]]
      atleticas_pontos_iguais.remove(team_ahead)

      # Nome da atlética que você deseja trocar
      team_behind = atleticas_pontos_iguais[0]

      # Encontrar a posição da atlética no DataFrame
      position_ahead = df_group.index[df_group['Time'] == team_ahead].tolist()[0]
      position_behind = df_group.index[df_group['Time'] == team_behind].tolist()[0]

      # Trocar os valores entre as linhas diretamente
      df_group.loc[position_ahead], df_group.loc[position_behind] = df_group.loc[position_behind].copy(), df_group.loc[position_ahead].copy()

    print(f"Classificação grupo {chr(idx + 64)} atualizada com confrontos diretos:")
    print(df_group)
    df_groups.append(df_group)
    idx = idx + 1
  return df_groups


# Simulação

In [12]:
def atualizar_df_games(new_game_data):
    global df_games
    df_games = pd.concat([df_games, pd.DataFrame([new_game_data])], ignore_index=True)

def remover_df_games(matches_to_remove, simulador=True):
    # Obter os índices dos jogos a serem removidos
    indices_to_remove = matches_to_remove.index

    # Remover os jogos do DataFrame original
    df_games.drop(indices_to_remove, inplace=True)

# Simular o resultado do jogo (exemplo simples)
def simular_jogo(group, home_team, home_goal, away_team, away_goal, confrontos_diretos):
    
    if listar_confrontos(home_team, away_team).empty == False:
        print("Não posso substituir um jogo que já existe")
    else:
        game = {
            'GRUPO': group,
            'EQUIPE Mandante': home_team,
            'GOLS_MANDANTE': home_goal,
            'EQUIPE Visitante': away_team,
            'GOLS_VISITANTE': away_goal,
            'PLACAR': str(home_goal) + 'x' + str(away_goal),
            'SIMULADOR': 1
        }

        # Definir df por grupo
        if group == 'A':
          df_group = df_groupA
        elif group == 'B':
          df_group = df_groupB

        condition_home = df_group['Atléticas'] == home_team
        condition_away = df_group['Atléticas'] == away_team

    # Atualizar o número de jogos
        df_group.loc[condition_home, 'Jogos'] += 1
        df_group.loc[condition_away, 'Jogos'] += 1

    # Atualizar os gols
        df_group.loc[condition_home, 'Gols Pró'] += home_goal
        df_group.loc[condition_home, 'Gols Contra'] += away_goal

        df_group.loc[condition_away, 'Gols Pró'] += away_goal
        df_group.loc[condition_away, 'Gols Contra'] += home_goal

        df_group.loc[condition_home, 'Saldo'] += home_goal - away_goal
        df_group.loc[condition_away, 'Saldo'] += away_goal - home_goal

    # Atualizar os pontos
        if home_goal == away_goal:
            df_group.loc[condition_home, 'Pontos'] += 1
            df_group.loc[condition_away, 'Pontos'] += 1
            df_group.loc[condition_home, 'E'] += 1
            df_group.loc[condition_away, 'E'] += 1
            update_confronto_direto(away_team, home_team, True, confrontos_diretos)
        elif home_goal > away_goal:
            df_group.loc[condition_home, 'Pontos'] += 3
            df_group.loc[condition_home, 'V'] += 1
            df_group.loc[condition_away, 'D'] += 1
            update_confronto_direto(home_team, away_team, False, confrontos_diretos)
        else:
            df_group.loc[condition_away, 'Pontos'] += 3
            df_group.loc[condition_away, 'V'] += 1
            df_group.loc[condition_home, 'D'] += 1
            update_confronto_direto(away_team, home_team, False, confrontos_diretos)

        atualizar_df_games(game)
        atualizar_classificacao(group, confrontos_to_df(confrontos_diretos))

In [ ]:
def remover_jogo(home_team, away_team, confrontos_diretos):

    df_confronto = listar_confrontos(home_team, away_team)
#   Se for jogo simulado  
    if df_confronto['SIMULADOR'].tolist()[0]:
        group = df_confronto['GRUPO'].values[0]

        # Definir df por grupo
        if group == 'A':
            df_group = df_groupA
        elif group == 'B':
            df_group = df_groupB

        home_team = df_confronto['EQUIPE Mandante'].tolist()[0]
        away_team = df_confronto['EQUIPE Visitante'].tolist()[0]
        home_goal = int(df_confronto['GOLS_MANDANTE'].tolist()[0])
        away_goal = int(df_confronto['GOLS_VISITANTE'].tolist()[0])

        condition_home = df_group['Atléticas'] == home_team
        condition_away = df_group['Atléticas'] == away_team

    # Atualizar o número de jogos
        df_group.loc[condition_home, 'Jogos'] -= 1
        df_group.loc[condition_away, 'Jogos'] -= 1

    # Atualizar os gols
        df_group.loc[condition_home, 'Gols Pró'] -= home_goal
        df_group.loc[condition_home, 'Gols Contra'] -= away_goal

        df_group.loc[condition_away, 'Gols Pró'] -= away_goal
        df_group.loc[condition_away, 'Gols Contra'] -= home_goal

        df_group.loc[condition_home, 'Saldo'] += home_goal - away_goal
        df_group.loc[condition_away, 'Saldo'] += away_goal - home_goal

    # Atualizar os pontos
        if home_goal == away_goal:
            df_group.loc[condition_home, 'Pontos'] -= 1
            df_group.loc[condition_away, 'Pontos'] -= 1
            df_group.loc[condition_home, 'E'] -= 1
            df_group.loc[condition_away, 'E'] -= 1
        elif home_goal > away_goal:
            df_group.loc[condition_home, 'Pontos'] -= 3
            df_group.loc[condition_home, 'V'] += 1
            df_group.loc[condition_away, 'D'] += 1
        else:
            df_group.loc[condition_away, 'Pontos'] -= 3
            df_group.loc[condition_away, 'V'] += 1
            df_group.loc[condition_home, 'D'] += 1

        remover_df_games(df_confronto)
        remover_confronto_direto(home_team, away_team, confrontos_diretos)
        atualizar_classificacao(group, confrontos_to_df(confrontos_diretos))
    else:
        print('Nenhum jogo simulado foi encontrado')

In [ ]:
import uuid

# Função para gerar IDs
def generate_game_id():
    return str(uuid.uuid4())

df_games['ID'] = [generate_game_id() for _ in range(len(df_games))]
df_games.rename(columns={'HORÁRIO': 'HORARIO'}, inplace=True)
df_games.rename(columns={'EQUIPE Mandante': 'Mandante'}, inplace=True)
df_games.rename(columns={'EQUIPE Visitante': 'Visitante'}, inplace=True)

In [ ]:
df_games.to_csv('files/games.csv', index=False)

In [ ]:
csv_file = 'files/FM.csv'
json_file = 'files/games.json'
csv_to_json(csv_file, json_file)

# Execução

In [ ]:
dic_modalities_page = {
    "FF/A" : "42-43",
    "FF/B" : "46-47",
    "FF/C" : "50-51",
    "FF/D" : "54-55",
    "FF/E" : "58-59",
    "FM/A" : "62-63",
    "FM/B" : "66-67",
    "FM/C" : "70-71",
    "FM/D" : "74-75",
    "FM/E" : "78-79",
    "FM/F" : "82-83",
}

In [ ]:
for key, value in dic_modalities_page.items():
    print(f"{key}: {value}")

In [ ]:
modality = 'FM/D'
tables = tabula.read_pdf("files/Boletim.pdf", pages=dic_modalities_page[modality])
print(tables)
tb_group = tables[0]
tb_games = tables[1]
tb_groups = [
    { 
        'group': 'A',
        'ranking': tables[3]
    },
    { 
        'group': 'B',
        'ranking': tables[4]
    }
    ]

## Zeradas

In [ ]:
filepath = 'files/' + modality
filepath_group = filepath + '/group'
rankings = create_zero_ranking_group(tb_group, filepath_group)
df_games = generate_table_games(tb_games)

### Dicionário de correção

In [ ]:
locations = ['Palestra', 'Idalina', 'Pinheiros', 'SEMEF', 'GETA', 'EDA', 'CESPRO']
correction_local = {
    'Pale stra': 'Palestra',
    'Idal ina': 'Idalina',
    'SEM EF': 'SEMEF',
}
# Função de validação e correção
def correct_local(local):
    if local in locations:
        return local
    elif local in correction_local:
        return correction_local[local]
    else:
        return local

# Aplicar a função de validação e correção nas colunas 'EQUIPE Mandante' e 'EQUIPE Visitante'
df_games['LOCAL'] = df_games['LOCAL'].apply(correct_local)
print(df_games['LOCAL'].unique())

In [ ]:
teams_groupA = tb_group['Grupo A'].tolist()
teams_groupB = tb_group['Grupo B'].tolist()
teams = teams_groupA + teams_groupB
correction_teams = {
    'ArquiteturaMackenzie': 'Arquitetura Mackenzie',
        'Belas A rtes': 'Belas Artes',
        'CAAP U FABC': 'CAAP UFABC',
        'CásperLíbero': 'Cásper Líbero',
        'Comunica ção PUC': 'Comunicação PUC',
        'Comunicaçã o Anhembi': 'Comunicação Anhembi',
        'Comunicaçã o Mackenzie': 'Comunicação Mackenzie',
        'Comunicaçã o Metodista': 'Comunicação Metodista',
        'Direit o PUC': 'Direito PUC',
        'Direit o USP': 'Direito USP',
        'Direito M ackenzie': 'Direito Mackenzie',
        'Direito S ã o Judas': 'Direito São Judas',
        'Direito S ão Judas': 'Direito São Judas',
        'Direito SãoBernardo': 'Direito São Bernardo',
        'DireitoFMU': 'Direito FMU',
        'EACHUSP': 'EACH USP',
        'Economia M ackenzie': 'Economia Mackenzie',
        'Economia M ackenzie': 'Economia Mackenzie',
        'Educação Fís i ca Anhembi': 'Educação Física Anhembi',
        'Educação Fís i ca UNINOVE': 'Educação Física UNINOVE',
        'Educação Fís ica Anhembi': 'Educação Física Anhembi',
        'EEFEUSP': 'EEFE USP',
        'EngenhariaAnhembi': 'Engenharia Anhembi',
        'EngenhariaMackenzie': 'Engenharia Mackenzie',
        'EngenhariaSão Judas': 'Engenharia São Judas',
        'EngenhariaUNICAMP': 'Engenharia UNICAMP',
        'ESP M': 'ESPM',
        'FAA P': 'FAAP',
        'Farmác ia USP': 'Farmácia USP',
        'Fatec Sã o Paulo': 'Fatec São Paulo',
        'FAUUSP': 'FAU USP',
        'FE I': 'FEI',
        'FEA P UC': 'FEA PUC',
        'FEA U SP': 'FEA USP',
        'FEC AP': 'FECAP',
        'Federaldo ABC': 'Federal do ABC',
        'FM U': 'FMU',
        'GetúlioVargas': 'Getúlio Vargas',
        'IBME C SP': 'IBMEC SP',
        'IME U SP': 'IME USP',
        'INS P ER': 'INSPER',
        'INSP ER': 'INSPER',
        'IT A': 'ITA',
        'LEP Mac kenzie': 'LEP Mackenzie',
        'LEP MAC KENZIE': 'LEP MACKENZIE',
        'Ma uá': 'Mauá',
        'Medici n a ABC': 'Medicina ABC',
        'Medici n a USP': 'Medicina USP',
        'Medicin a ABC': 'Medicina ABC',
        'Medicin a ABC': 'Medicina ABC',
        'Medicin a Santos': 'Medicina Santos',
        'Medicina PU C Campinas': 'Medicina PUC Campinas',
        'Medicina S ã o Caetano': 'Medicina São Caetano',
        'Medicina S anta Casa': 'Medicina Santa Casa',
        'Medicina Sa nto Amaro': 'Medicina Santo Amaro',
        'Medicina Sã o Bernardo': 'Medicina São Bernardo',
        'Medicina Sã o Caetano': 'Medicina São Caetano',
        'Medicina San  ta Marcelina': 'Medicina Santa Marcelina',
        'Medicina San ta Marcelina': 'Medicina Santa Marcelina',
        'Medicina Ta ubaté (DT)': 'Medicina Taubaté',
        'Medicina U NICAMP': 'Medicina UNICAMP',
        'Medicina U NINOVE': 'Medicina UNINOVE',
        'MedicinaAnhembi': 'Medicina Anhembi',
        'MedicinaBragança': 'Medicina Bragança',
        'MedicinaEinstein': 'Medicina Einstein',
        'MedicinaJundiaí': 'Medicina Jundiaí',
        'MedicinaPaulista': 'Medicina Paulista',
        'MedicinaTaubaté': 'Medicina Taubaté',
        'MedicinaUNICID': 'Medicina UNICID',
        'MedicinaUNIMES': 'Medicina UNIMES',
        'MedicinaZ Taubaté': 'Medicina Taubaté',
        'Politécn ica USP': 'Politécnica USP',
        'Psicologia  d a PUC SP': 'Psicologia da PUC SP',
        'Psicologia d a PUC SP': 'Psicologia da PUC SP',
        'SEN  AC': 'SENAC',
        'SEN AC': 'SENAC',
        'Sistema de Inf ormação USP': 'Sistema de Informação USP',
        'UN IP': 'UNIP',
        'UNIFESP D iadema': 'UNIFESP Diadema',
        'Unifesp S ão Paulo': 'Unifesp São Paulo',
        'UNIFESPOsasco': 'UNIFESP Osasco',
        'US C S': 'USCS',
        'US CS': 'USCS',
        'Veteriná ria USP': 'Veterinária USP',
}
# Função de validação e correção
def correct_team(equipe):
    equipe_strip = equipe.strip().replace('  ', ' ')
    if equipe_strip in teams:
        return equipe_strip
    elif equipe_strip in correction_teams:
        return correction_teams[equipe_strip]
    else:
        return equipe_strip

# Aplicar a função de validação e correção nas colunas 'EQUIPE Mandante' e 'EQUIPE Visitante'
df_games['Mandante'] = df_games['Mandante'].apply(correct_team)
df_games['Visitante'] = df_games['Visitante'].apply(correct_team)
print(df_games['Mandante'].unique())
print(df_games['Visitante'].unique())

In [ ]:
create_games_files(df_games, filepath)
gerar_confronto_direto(df_games, filepath)

## Atualização

In [ ]:
# confrontos = gerar_confronto_direto(df_games, filepath)
confrontos = load_json_data('files/FM/A/confrontation.json')
df_confrontos_diretos = confrontos_to_df(confrontos)
atualizar_classificacao(tb_groups, df_confrontos_diretos)

In [7]:
tables = tabula.read_pdf("files/Boletim.pdf", pages="62-64")
print(tables)
# tb_group = tables[0]
# tb_games = tables[1]
# tb_groups = [
#     { 
#         'group': 'A',
#         'ranking': tables[3]
#     },
#     { 
#         'group': 'B',
#         'ranking': tables[4]
#     }
#     ]

# filepath = 'files/' + modality
# # filepath_group = filepath + '/group'
# df_games = generate_table_games(tb_games)
# df_games = corrigir_local(df_games)
# df_games = corrigir_times(tb_group, df_games)
# create_games_files(df_games, filepath)
# gerar_confronto_direto(df_games, filepath)
# confrontos = gerar_confronto_direto(df_games, filepath)
# df_confrontos_diretos = confrontos_to_df(confrontos)
# atualizar_classificacao(tb_groups, df_confrontos_diretos)

[             Unnamed: 0      Unnamed: 1
0               Grupo A         Grupo B
1        Federal do ABC          INSPER
2                 FECAP        EEFE USP
3  Engenharia São Judas   FEA São Judas
4                   FEI  Getúlio Vargas
5               FEA USP         FEA PUC
6                  ESPM     Direito PUC,        DIA    HORÁRIO      LOCAL GRUPO        EQUIPE Mandante PLACAR  \
0    02/03   10h40min    Idalina     B            Direito PUC      X   
1   03/ 03  09h20 min   Idal ina     A  Engenharia  São Judas      X   
2    03/03   10h40min    Idalina     A                    FEI      X   
3    03/03   17h20min    Idalina     B               EEFE USP      X   
4   09/ 03  11h10 min  Pale stra     A        Federal  do ABC      X   
5   10/ 03       16 h   Idal ina     B                INSP ER      X   
6   16/ 03  09h50 min  Pale stra     A          Federaldo ABC      X   
7   17/ 03  10h40 min   Idal ina     A                  ESP M      X   
8    17/03        12h    Idali

In [8]:
tables[3]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Gols,Gols.1,Unnamed: 7
0,Col.,Grupo A,Pontos,Jogos,Vitórias,Empates,Derrotas,NaN,NaN,Saldo
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Pró,Contra,NaN
2,1o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,5o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,6o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
tables = tabula.read_pdf("files/Boletim.pdf", pages="58-60")
# tables[3][2][1]

Error importing jpype dependencies. Fallback to subprocess.
No module named 'jpype'


In [10]:
tb_groupA = tables[3]

In [15]:
df_groupA = pd.DataFrame(tb_groupA)

In [18]:

print(df_groupA)

  Unnamed: 0            Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4  \
0       Col.               Grupo A     Pontos      Jogos   Vitórias   
1        NaN                   NaN        NaN        NaN        NaN   
2         1o       Medicina UNICID         03         01         01   
3         2o           Direito FMU         03         01         01   
4         3o     Medicina Bragança         00         00         00   
5         4o  Medicina São Caetano         00         00         00   
6         5o                   NaN        NaN        NaN        NaN   
7         6o                   NaN        NaN        NaN        NaN   
8         7o                   NaN        NaN        NaN        NaN   

  Unnamed: 5 Unnamed: 6 Gols  Gols.1 Unnamed: 7  
0    Empates   Derrotas  NaN     NaN      Saldo  
1        NaN        NaN  Pró  Contra        NaN  
2         00         00   03      00         03  
3         00         00   01      00         01  
4         00         01   00      01   

In [20]:
df_groupA.iloc[2, 1]

'Medicina UNICID'

In [ ]:
def tem_ranking(df_group):
    elemento = df_group.iloc[2, 1]
    return pd.isna(elemento) == False

In [13]:
def atualizar_dados_times(df_nova, df_original):
    # Salvar a coluna 'ID' em uma variável temporária
    id_column = df_original.pop('ID')

    # Redefinir o índice do DataFrame df_original
    df_original.reset_index(inplace=True)

    # Atualizar os dados em df_original com base em df_nova
    for index, row in df_nova.iterrows():
        time = row['Time']
        idx = df_original[df_original['Time'] == time].index
        if not idx.empty:
            df_original.loc[idx[0]] = row

    # Reatribuir a coluna 'ID' ao DataFrame df_original
    df_original['ID'] = id_column

    # Agora, df_original contém os dados atualizados
    print(df_original)
    return df_original

In [11]:
new_df_groupA = format_table_group([tb_groupA])[0]
print(new_df_groupA)
data_original = load_json_data('files/FF/E/group/ranking_A.json')
df_original = pd.DataFrame(data_original)
print(df_original)

                   Time  Pontos  Jogos  V  E  D  Gols_Pro  Gols_Contra  Saldo
2       Medicina UNICID       3      1  1  0  0         3            0      3
3           Direito FMU       3      1  1  0  0         1            0      1
4     Medicina Bragança       0      0  0  0  1         0            1     -1
5  Medicina São Caetano       0      0  0  0  1         0            3     -3
6                     0       0      0  0  0  0         0            0      0
7                     0       0      0  0  0  0         0            0      0
8                     0       0      0  0  0  0         0            0      0
                   Time Pontos Jogos  V  E  D Gols_Pro Gols_Contra Saldo  \
0               FAU USP      0     0  0  0  0        0           0     0   
1  Medicina São Caetano      0     0  0  0  0        0           0     0   
2     Medicina Bragança      0     0  0  0  0        0           0     0   
3           Direito FMU      0     0  0  0  0        0           0     0

In [14]:
print(new_df_groupA.columns)
print(df_original.columns)
df_atualizado = atualizar_dados_times(new_df_groupA, df_original)
tb_teste = [{
    'group': 'A',
    'ranking': df_atualizado
}]

Index(['Time', 'Pontos', 'Jogos', 'V', 'E', 'D', 'Gols_Pro', 'Gols_Contra',
       'Saldo'],
      dtype='object')
Index(['Time', 'Pontos', 'Jogos', 'V', 'E', 'D', 'Gols_Pro', 'Gols_Contra',
       'Saldo', 'ID'],
      dtype='object')
   index                  Time Pontos Jogos  V  E  D Gols_Pro Gols_Contra  \
0    0.0               FAU USP      0     0  0  0  0        0           0   
1    NaN  Medicina São Caetano      0     0  0  0  1        0           3   
2    NaN     Medicina Bragança      0     0  0  0  1        0           1   
3    NaN           Direito FMU      3     1  1  0  0        1           0   
4    4.0                  USCS      0     0  0  0  0        0           0   
5    NaN       Medicina UNICID      3     1  1  0  0        3           0   
6    6.0       Comunicação PUC      0     0  0  0  0        0           0   

  Saldo                                    ID  
0     0  effcafd2-9063-4045-846c-669205b22934  
1    -3  bfcbb242-b087-474e-b696-9ba3ce16f70d  
2  

In [107]:
tb_teste[0]['ranking'].columns

Index(['index', 'Time', 'Pontos', 'Jogos', 'V', 'E', 'D', 'Gols_Pro',
       'Gols_Contra', 'Saldo', 'ID'],
      dtype='object')

In [15]:
confrontos = load_json_data('files/FM/A/confrontation.json')
df_confrontos_diretos = confrontos_to_df(confrontos)
# df_updated = atualizar_classificacao(tb_teste, df_confrontos_diretos)

Classificação grupo A atualizada com confrontos diretos:
None


In [18]:
tb_teste[0]['ranking']

,index,Time,Pontos,Jogos,V,E,D,Gols_Pro,Gols_Contra,Saldo,ID
0,0.0,FAU USP,0,0,0,0,0,0,0,0,effcafd2-9063-4045-846c-669205b22934
1,NaN,Medicina São Caetano,0,0,0,0,1,0,3,-3,bfcbb242-b087-474e-b696-9ba3ce16f70d
2,NaN,Medicina Bragança,0,0,0,0,1,0,1,-1,d3fe404a-a182-45c0-8466-e227c8ee2396
3,NaN,Direito FMU,3,1,1,0,0,1,0,1,31dbc394-becb-47ec-bd74-5a72dc48ae90
4,4.0,USCS,0,0,0,0,0,0,0,0,edbae3fa-f358-455f-9fcb-424355c60ade
5,NaN,Medicina UNICID,3,1,1,0,0,3,0,3,aa11b31f-ac0e-4405-98b9-60cf03c6d19a
6,6.0,Comunicação PUC,0,0,0,0,0,0,0,0,2034d9ef-7bba-4a7e-9a62-f5cf175c5b83


In [19]:
def get_rankings_zero_group(modality):
    ranking_A = load_json_data(f'files/{modality}/group/ranking_zero_A.json')
    ranking_B = load_json_data(f'files/{modality}/group/ranking_zero_B.json')
    return {
        'A': ranking_A,
        'B': ranking_B
    }

In [21]:
def get_index_teams(rankings):
    ranking_dict_A = {time_dict['Time']: index for index, time_dict in enumerate(rankings['A'])}
    ranking_dict_B = {time_dict['Time']: index for index, time_dict in enumerate(rankings['B'])}
    return {
        'A': ranking_dict_A,
        'B': ranking_dict_B
    }

In [17]:
def generate_ranking_by_games(modality):
    
    json_games = load_json_data(f'files/{modality}/games.json')
    rankings = get_rankings_zero_group(modality)
    index_teams = get_index_teams(rankings)

    # Percorra cada jogo no DataFrame df_games
    for jogo in json_games:
        if jogo["PLACAR"] == "X":
            continue
        grupo = jogo["GRUPO"]
        mandante = jogo["Mandante"]
        visitante = jogo["Visitante"]
        dados_ranking_mandante = rankings[grupo][index_teams[grupo][mandante]]
        dados_ranking_visitante = rankings[grupo][index_teams[grupo][visitante]]
        placar_mandante = int(jogo["GOLS_MANDANTE"]) 
        placar_visitante = int(jogo["GOLS_VISITANTE"])
        
        # Atualize os pontos das equipes com base no resultado do jogo
        if placar_mandante > placar_visitante:
            dados_ranking_mandante['Pontos'] = int(dados_ranking_mandante['Pontos']) + 3
            dados_ranking_mandante['V'] = int(dados_ranking_mandante['V']) + 1
            dados_ranking_visitante['D'] = int(dados_ranking_visitante['D']) + 1
            
        
        elif placar_mandante == placar_visitante:
            dados_ranking_mandante['Pontos'] = int(dados_ranking_mandante['Pontos']) + 1
            dados_ranking_visitante['Pontos'] = int(dados_ranking_visitante['Pontos']) + 1
            dados_ranking_mandante['E'] = int(dados_ranking_mandante['E']) + 1
            dados_ranking_visitante['E'] = int(dados_ranking_visitante['E']) + 1
            
        else:
            dados_ranking_visitante['Pontos'] = int(dados_ranking_visitante['Pontos']) + 3
            dados_ranking_visitante['V'] = int(dados_ranking_visitante['V']) + 1
            dados_ranking_mandante['D'] = int(dados_ranking_mandante['D']) + 1

        dados_ranking_mandante['Jogos'] = int(dados_ranking_mandante['Jogos']) + 1
        dados_ranking_visitante['Jogos'] = int(dados_ranking_visitante['Jogos']) + 1
        
        
        dados_ranking_mandante['Gols_Pro'] = int(dados_ranking_mandante['Gols_Pro']) + placar_mandante
        dados_ranking_visitante['Gols_Pro'] = int(dados_ranking_visitante['Gols_Pro']) + placar_visitante
        
        dados_ranking_mandante['Gols_Contra'] = int(dados_ranking_mandante['Gols_Contra']) + placar_visitante
        dados_ranking_visitante['Gols_Contra'] = int(dados_ranking_visitante['Gols_Contra']) + placar_mandante

        dados_ranking_mandante['Saldo'] = int(dados_ranking_mandante['Gols_Pro']) - int(dados_ranking_mandante['Gols_Contra'])
        dados_ranking_visitante['Saldo'] = int(dados_ranking_visitante['Gols_Pro']) - int(dados_ranking_visitante['Gols_Contra'])
        
    return rankings

In [13]:
def corrigir_local(df_games):
    locations = ['Palestra', 'Idalina', 'Pinheiros', 'SEMEF', 'GETA', 'EDA', 'CESPRO']
    correction_local = {
        'Pale stra': 'Palestra',
        'Idal ina': 'Idalina',
        'SEM EF': 'SEMEF',
    }
    # Função de validação e correção
    def correct_local(local):
        if local in locations:
            return local
        elif local in correction_local:
            return correction_local[local]
        else:
            return local

    # Aplicar a função de validação e correção nas colunas 'EQUIPE Mandante' e 'EQUIPE Visitante'
    df_games['LOCAL'] = df_games['LOCAL'].apply(correct_local)
    print('LOCAL:')
    print(df_games['LOCAL'].unique())
    return df_games

In [14]:
def corrigir_times(tb_group, df_games): 
    teams_groupA = tb_group['Grupo A'].tolist()
    teams_groupB = tb_group['Grupo B'].tolist()
    teams = teams_groupA + teams_groupB
    correction_teams = {
        'ArquiteturaMackenzie': 'Arquitetura Mackenzie',
        'Belas A rtes': 'Belas Artes',
        'CAAP U FABC': 'CAAP UFABC',
        'Comunica ção PUC': 'Comunicação PUC',
        'Comunicaçã o Anhembi': 'Comunicação Anhembi',
        'Comunicaçã o Mackenzie': 'Comunicação Mackenzie',
        'Comunicaçã o Metodista': 'Comunicação Metodista',
        'Direit o PUC': 'Direito PUC',
        'Direit o USP': 'Direito USP',
        'Direito M ackenzie': 'Direito Mackenzie',
        'Direito S ã o Judas': 'Direito São Judas',
        'Direito S ão Judas': 'Direito São Judas',
        'Direito SãoBernardo': 'Direito São Bernardo',
        'EACHUSP': 'EACH USP',
        'Economia M ackenzie': 'Economia Mackenzie',
        'Economia M ackenzie': 'Economia Mackenzie',
        'Educação Fís i ca Anhembi': 'Educação Física Anhembi',
        'Educação Fís i ca UNINOVE': 'Educação Física UNINOVE',
        'Educação Fís ica Anhembi': 'Educação Física Anhembi',
        'EngenhariaAnhembi': 'Engenharia Anhembi',
        'EngenhariaMackenzie': 'Engenharia Mackenzie',
        'EngenhariaSão Judas': 'Engenharia São Judas',
        'ESP M': 'ESPM',
        'Farmác ia USP': 'Farmácia USP',
        'Fatec Sã o Paulo': 'Fatec São Paulo',
        'FAUUSP': 'FAU USP',
        'FE I': 'FEI',
        'FEA P UC': 'FEA PUC',
        'FEA U SP': 'FEA USP',
        'FEC AP': 'FECAP',
        'Federaldo ABC': 'Federal do ABC',
        'FM U': 'FMU',
        'GetúlioVargas': 'Getúlio Vargas',
        'IBME C SP': 'IBMEC SP',
        'INS P ER': 'INSPER',
        'INSP ER': 'INSPER',
        'IT A': 'ITA',
        'LEP Mac kenzie': 'LEP Mackenzie',
        'LEP MAC KENZIE': 'LEP MACKENZIE',
        'Medici n a USP': 'Medicina USP',
        'Medicin a Santos': 'Medicina Santos',
        'Medicina PU C Campinas': 'Medicina PUC Campinas',
        'Medicina S ã o Caetano': 'Medicina São Caetano',
        'Medicina S anta Casa': 'Medicina Santa Casa',
        'Medicina Sa nto Amaro': 'Medicina Santo Amaro',
        'Medicina Sã o Bernardo': 'Medicina São Bernardo',
        'Medicina Sã o Caetano': 'Medicina São Caetano',
        'Medicina San  ta Marcelina': 'Medicina Santa Marcelina',
        'Medicina San ta Marcelina': 'Medicina Santa Marcelina',
        'Medicina Ta ubaté (DT)': 'Medicina Taubaté',
        'Medicina U NICAMP': 'Medicina UNICAMP',
        'MedicinaAnhembi': 'Medicina Anhembi',
        'MedicinaBragança': 'Medicina Bragança',
        'MedicinaEinstein': 'Medicina Einstein',
        'MedicinaJundiaí': 'Medicina Jundiaí',
        'MedicinaTaubaté': 'Medicina Taubaté',
        'MedicinaUNICID': 'Medicina UNICID',
        'MedicinaUNIMES': 'Medicina UNIMES',
        'MedicinaZ Taubaté': 'Medicina Taubaté',
        'Politécn ica USP': 'Politécnica USP',
        'Psicologia  d a PUC SP': 'Psicologia da PUC SP',
        'Psicologia d a PUC SP': 'Psicologia da PUC SP',
        'SEN  AC': 'SENAC',
        'SEN AC': 'SENAC',
        'Sistema de Inf ormação USP': 'Sistema de Informação USP',
        'UN IP': 'UNIP',
        'UNIFESP D iadema': 'UNIFESP Diadema',
        'Unifesp S ão Paulo': 'Unifesp São Paulo',
        'UNIFESPOsasco': 'UNIFESP Osasco',
        'US C S': 'USCS',
        'US CS': 'USCS',
        'Veteriná ria USP': 'Veterinária USP',
    }
    # Função de validação e correção
    def correct_team(equipe):
        equipe_strip = equipe.strip().replace('  ', ' ')
        if equipe_strip in teams:
            return equipe_strip
        elif equipe_strip in correction_teams:
            return correction_teams[equipe_strip]
        else:
            return equipe_strip

    # Aplicar a função de validação e correção nas colunas 'EQUIPE Mandante' e 'EQUIPE Visitante'
    df_games['Mandante'] = df_games['Mandante'].apply(correct_team)
    df_games['Visitante'] = df_games['Visitante'].apply(correct_team)
    print('Mandante:')
    print(df_games['Mandante'].unique())
    print('Visitante:')
    print(df_games['Visitante'].unique())
    return df_games

In [5]:
def format_tb_group(tb_group):
    if list(tb_group.columns) != ['Grupo A', 'Grupo B']:
        # Definir a primeira linha como cabeçalho da tabela
        tb_group.columns = tb_group.iloc[0]
        # Remover a primeira linha, que agora é o cabeçalho
        return tb_group[1:]

In [39]:
modality = 'FM/C'
pages = '70-71'
print(f"modalidade: {modality}")
# tables = tabula.read_pdf("files/Boletim.pdf", pages=dic_modalities_page[modality])
tables = tabula.read_pdf("files/Boletim.pdf", pages=pages)

modalidade: FM/C


In [28]:
table_two = tables[2]
ultimo_elemento_df = table_two.tail(1)
print(ultimo_elemento_df['EQUIPE Mandante'])

   DIA HORÁRIO                                        LOCAL GRUPO  \
9  NaN     NaN  Jogos adiados devido falta de energia elétrica...   

  EQUIPE Mandante PLACAR Unnamed: 0 EQUIPE Visitante  
9             NaN    NaN        NaN              NaN  


In [29]:
print(ultimo_elemento_df['EQUIPE Mandante'].isna())

9    True
Name: EQUIPE Mandante, dtype: bool


In [40]:
tb_group = format_tb_group(tables[0])
# tb_games = pd.concat([tables[1], tables[2]], ignore_index=True)
# print(tb_group)

# filepath = 'files/' + modality

df_games = generate_table_games([tables[1], tables[2]])
print(df_games)
df_games = corrigir_local(df_games)
df_games = corrigir_times(tb_group, df_games)


          DIA    HORARIO      LOCAL GRUPO                   Mandante PLACAR  \
0  2024-02-25   08h30min      SEMEF     A      Medicina São Bernardo  03X01   
1  2024-02-25   09h50min      SEMEF     A          Medicina Einstein  01X03   
2  2024-02-25   19h10min      SEMEF     B                       Mauá  00X04   
3  2024-02-25   20h30min      SEMEF     B              Cásper Líbero  03X03   
4  2024-03-02  13h20 min   Idal ina     A   Educação Fís ica Anhembi  04X01   
5  2024-03-03  14h40 min   Idal ina     B          Medicina  Jundiaí  02X03   
6  2024-03-09  09h50 min  Pale stra     B                    SEN  AC  02X05   
7  2024-03-10  13h20 min   Idal ina     A           Medicina  Santos  04X05   
8  2024-03-17       16 h   Idal ina     B                      Ma uá      X   
9  2024-03-17   17h20min    Idalina     A          Direito Mackenzie      X   
10 2024-03-23       19 h     SEM EF     B          Medicina  UNINOVE      X   
11 2024-03-24  09h20 min   Idal ina     A   Educação

In [24]:
tb_games.columns

Index(['DIA', 'HORÁRIO', 'LOCAL', 'GRUPO', 'EQUIPE Mandante', 'PLACAR',
       'EQUIPE Visitante', 'LOCAL GRUPO', 'Unnamed: 0'],
      dtype='object')

In [22]:
common_columns = tables[1].columns.intersection(tables[2].columns)
print(common_columns)

Index(['DIA', 'HORÁRIO', 'EQUIPE Mandante', 'PLACAR', 'EQUIPE Visitante'], dtype='object')


In [98]:
import numpy as np

def find_index_duplicated(data_table):
    # Extração dos pontos de cada equipe
    pontos = np.array([int(entry['Pontos']) for entry in data_table])

    # Identificação dos índices com pontos duplicados
    _, indices_pontos_duplicados = np.unique(pontos, return_index=True)
    indices_pontos_duplicados = np.sort(indices_pontos_duplicados)

    # Retornar os índices das entradas com pontos duplicados
    print(indices_pontos_duplicados)
    return indices_pontos_duplicados

In [131]:
tb_group = rankings['A']
columns_to_convert = ['Pontos', 'Jogos', 'V', 'E', 'D', 'Gols_Pro', 'Gols_Contra', 'Saldo']

df_group = pd.DataFrame(tb_group)
df_group[columns_to_convert] = df_group[columns_to_convert].astype(int)
df_group.sort_values(by=['Pontos', 'Saldo', 'Gols_Pro'], ascending=[False, False, False], inplace=True)

df_group.reset_index(drop=True, inplace=True)

print('Antes:', df_group)

pontos_iguais = df_group['Pontos'].unique().tolist()

print('pontos_iguais', pontos_iguais)

# Obter apenas os nomes das atléticas
for ponto in pontos_iguais:
    atléticas_pontos_iguais = df_group[df_group['Pontos'] == ponto]['Time'].tolist()
    if len(atléticas_pontos_iguais) == 2:
      print('atléticas_pontos_iguais:', atléticas_pontos_iguais)
      team_ahead = df_confrontos_diretos.loc[atléticas_pontos_iguais[0], atléticas_pontos_iguais[1]]
      print('team_ahead', team_ahead)
      if team_ahead != 'E':
        atléticas_pontos_iguais.remove(team_ahead)

        # Nome da atlética que você deseja trocar
        team_behind = atléticas_pontos_iguais[0]

        # Encontrar a posição da atlética no DataFrame
        position_ahead = df_group.index[df_group['Time'] == team_ahead].tolist()[0]
        position_behind = df_group.index[df_group['Time'] == team_behind].tolist()[0]
        print('posições', position_ahead, position_behind )
        if position_ahead > position_behind:
            # Trocar os valores entre as linhas diretamente
            df_group.loc[position_ahead], df_group.loc[position_behind] = df_group.loc[position_behind].copy(), df_group.loc[position_ahead].copy()
            print('Atualizado:', df_group)

Antes:                    Time  Pontos  Jogos  V  E  D  Gols_Pro  Gols_Contra  Saldo  \
0           Direito FMU       3      1  1  0  0         4            0      4   
1       Medicina UNICID       3      1  1  0  0         3            0      3   
2               FAU USP       0      0  0  0  0         0            0      0   
3                  USCS       0      0  0  0  0         0            0      0   
4       Comunicação PUC       0      0  0  0  0         0            0      0   
5     Medicina Bragança       0      1  0  0  1         0            1     -1   
6  Medicina São Caetano       0      1  0  0  1         0            3     -3   

                                     ID  
0  31dbc394-becb-47ec-bd74-5a72dc48ae90  
1  aa11b31f-ac0e-4405-98b9-60cf03c6d19a  
2  effcafd2-9063-4045-846c-669205b22934  
3  edbae3fa-f358-455f-9fcb-424355c60ade  
4  2034d9ef-7bba-4a7e-9a62-f5cf175c5b83  
5  d3fe404a-a182-45c0-8466-e227c8ee2396  
6  bfcbb242-b087-474e-b696-9ba3ce16f70d  
pontos_i

In [128]:
df_group['Pontos'].unique().tolist()

[3, 0]

In [119]:
def update_ranking(tb_groups, df_confrontos_diretos):
  df_groups = []
  idx = 1
  columns_to_convert = ['Pontos', 'Jogos', 'V', 'E', 'D', 'Gols_Pro', 'Gols_Contra', 'Saldo']
  
  for tb_group in tb_groups: 
    df_group = pd.DataFrame(tb_group)
    df_group[columns_to_convert] = df_group[columns_to_convert].astype(int)
    df_group.sort_values(by=['Pontos', 'Saldo', 'Gols_Pro'], ascending=[False, False, False], inplace=True)
    df_group.reset_index(drop=True, inplace=True)
    pontos_iguais = df_group['Pontos'].unique().tolist()

    for ponto in pontos_iguais:
      times_pontos_iguais = df_group[df_group['Pontos'] == ponto]['Time'].tolist()
      if len(times_pontos_iguais) == 2:
        print('times_pontos_iguais:', times_pontos_iguais)
        team_ahead = df_confrontos_diretos.loc[times_pontos_iguais[0], times_pontos_iguais[1]]
        print('team_ahead', team_ahead)
        if team_ahead != 'E':
          times_pontos_iguais.remove(team_ahead)

          # Nome da atlética que você deseja trocar
          team_behind = times_pontos_iguais[0]

          # Encontrar a posição da atlética no DataFrame
          position_ahead = df_group.index[df_group['Time'] == team_ahead].tolist()[0]
          position_behind = df_group.index[df_group['Time'] == team_behind].tolist()[0]
          print('posições', position_ahead, position_behind )
          if position_ahead > position_behind:
              # Trocar os valores entre as linhas diretamente
              df_group.loc[position_ahead], df_group.loc[position_behind] = df_group.loc[position_behind].copy(), df_group.loc[position_ahead].copy()
              
    df_groups.append(df_group)
    idx = idx + 1
  return df_groups


In [11]:
def update_ranking_group(tb_group, df_confrontos_diretos):
    columns_to_convert = ['Pontos', 'Jogos', 'V', 'E', 'D', 'Gols_Pro', 'Gols_Contra', 'Saldo']
  
    df_group = pd.DataFrame(tb_group)
    df_group[columns_to_convert] = df_group[columns_to_convert].astype(int)
    df_group.sort_values(by=['Pontos', 'Saldo', 'Gols_Pro'], ascending=[False, False, False], inplace=True)
    df_group.reset_index(drop=True, inplace=True)
    pontos_iguais = df_group['Pontos'].unique().tolist()

    for ponto in pontos_iguais:
        times_pontos_iguais = df_group[df_group['Pontos'] == ponto]['Time'].tolist()
        if len(times_pontos_iguais) == 2:
            print('times_pontos_iguais:', times_pontos_iguais)
            team_ahead = df_confrontos_diretos.loc[times_pontos_iguais[0], times_pontos_iguais[1]]
            print('team_ahead', team_ahead)
            if team_ahead != 'E':
                times_pontos_iguais.remove(team_ahead)

                # Nome da atlética que você deseja trocar
                team_behind = times_pontos_iguais[0]

                # Encontrar a posição da atlética no DataFrame
                position_ahead = df_group.index[df_group['Time'] == team_ahead].tolist()[0]
                position_behind = df_group.index[df_group['Time'] == team_behind].tolist()[0]
                print('posições', position_ahead, position_behind )
                if position_ahead > position_behind:
                    # Trocar os valores entre as linhas diretamente
                    df_group.loc[position_ahead], df_group.loc[position_behind] = df_group.loc[position_behind].copy(), df_group.loc[position_ahead].copy()
              
    return df_group


In [118]:
df_confrontos_diretos.loc['Direito FMU', 'Medicina UNICID'] = 'Direito FMU'
df_confrontos_diretos.loc['Medicina UNICID', 'Direito FMU'] = 'Direito FMU'

In [117]:
df_confrontos_diretos.loc['Direito FMU', 'Medicina UNICID']

'Direito FMU'

In [59]:
df_groups = update_ranking([rankings['A'], rankings['B']], df_confrontos_diretos)

linhas_pontos_iguais
                   Time Pontos Jogos  V  E  D Gols_Pro Gols_Contra Saldo  \
0               FAU USP      0     0  0  0  0        0           0     0   
1  Medicina São Caetano      0     1  0  0  1        0           3    -3   
2     Medicina Bragança      0     1  0  0  1        0           1    -1   
3           Direito FMU      3     1  1  0  0        1           0     1   
4                  USCS      0     0  0  0  0        0           0     0   
5       Medicina UNICID      3     1  1  0  0        3           0     3   
6       Comunicação PUC      0     0  0  0  0        0           0     0   

                                     ID  
0  effcafd2-9063-4045-846c-669205b22934  
1  bfcbb242-b087-474e-b696-9ba3ce16f70d  
2  d3fe404a-a182-45c0-8466-e227c8ee2396  
3  31dbc394-becb-47ec-bd74-5a72dc48ae90  
4  edbae3fa-f358-455f-9fcb-424355c60ade  
5  aa11b31f-ac0e-4405-98b9-60cf03c6d19a  
6  2034d9ef-7bba-4a7e-9a62-f5cf175c5b83  
linhas_pontos_iguais
             

In [15]:
def update_game(modality, game_id, home_goal, away_goal):
    file_path = f'files/{modality}/games.json'
    games_data = load_json_data(file_path)

    # Encontrar o jogo com o ID fornecido
    for game in games_data:
        if game['ID'] == game_id:
            # Atualizar o placar, gols do mandante e gols do visitante
            group = game['GRUPO']
            game['PLACAR'] = f"{home_goal}X{away_goal}"
            game['GOLS_MANDANTE'] = str(home_goal)
            game['GOLS_VISITANTE'] = str(away_goal)
            home_team = game['Mandante'] 
            away_team = game['Visitante']
            winner_team = home_team
            loser_team = away_team
            draw = False
            if int(home_goal) < int(away_goal):
                winner_team = away_team
                loser_team = home_team
            elif home_goal == away_goal: 
                draw = True
            break
    
    file_path_group = f'files/{modality}/group'
    save_json_data(games_data, file_path)
    confrontos_diretos = load_json_data(f'files/{modality}/confrontation.json')
    df_confrontos_diretos = confrontos_to_df(update_confronto_direto(winner_team, loser_team, draw, confrontos_diretos))
    rankings = generate_ranking_by_games(modality)
    df_groups = update_ranking([rankings[group]], df_confrontos_diretos)

    df_group = df_groups[0]
    df_group.to_csv(f'{file_path_group}/ranking_{group}.csv', index=False)

    csv_to_json(f'{file_path_group}/ranking_{group}.csv', f'{file_path_group}/ranking_{group}.json')
    

In [41]:
def create_game(modality, home_team, away_team, date, time, venue, group, home_goal, away_goal, simulator=False):
    # Atualizar o arquivo JSON de jogos
    file_path = f'files/{modality}/games.json'
    games_data = load_json_data(file_path)

    # Criar um novo jogo
    new_game = {
        "DIA": date,
        "HORARIO": time,
        "LOCAL": venue,
        "GRUPO": group,
        "Mandante": home_team,
        "PLACAR": f"{home_goal}X{away_goal}",
        "Visitante": away_team,
        "GOLS_MANDANTE": str(home_goal),
        "GOLS_VISITANTE": str(away_goal),
        "SIMULADOR": simulator,  # Defina como necessário
        "ID": generate_game_id()
    }

    # Adicionar o novo jogo à lista de jogos
    games_data.append(new_game)

    # Salvar os dados atualizados
    save_json_data(games_data, file_path)

    # Atualizar o arquivo CSV de jogos
    df_new_game = pd.DataFrame([new_game])
    csv_file_path = f'files/{modality}/games.csv'
    if os.path.exists(csv_file_path):
        df_existing_games = pd.read_csv(csv_file_path)
        df_updated_games = pd.concat([df_existing_games, df_new_game], ignore_index=True)
    else:
        df_updated_games = df_new_game

    # Salvar o DataFrame atualizado como um arquivo CSV
    df_updated_games.to_csv(csv_file_path, index=False)

    # Converter o arquivo CSV em JSON
    csv_to_json(csv_file_path, file_path)

In [2]:
def create_zero_game(modality, home_team, away_team, group, simulator=True):
    # Atualizar o arquivo JSON de jogos
    file_path = f'files/{modality}/games.json'
    games_data = load_json_data(file_path)

    # Criar um novo jogo
    new_game = {
        "DIA": '',
        "HORARIO": '',
        "LOCAL": '',
        "GRUPO": group,
        "Mandante": home_team,
        "PLACAR": "X",
        "Visitante": away_team,
        "GOLS_MANDANTE": '',
        "GOLS_VISITANTE": '',
        "SIMULADOR": simulator,  # Defina como necessário
        "ID": generate_game_id()
    }

    # Adicionar o novo jogo à lista de jogos
    games_data.append(new_game)

    # Salvar os dados atualizados
    save_json_data(games_data, file_path)

    # Atualizar o arquivo CSV de jogos
    df_new_game = pd.DataFrame([new_game])
    csv_file_path = f'files/{modality}/games.csv'
    if os.path.exists(csv_file_path):
        df_existing_games = pd.read_csv(csv_file_path)
        df_updated_games = pd.concat([df_existing_games, df_new_game], ignore_index=True)
    else:
        df_updated_games = df_new_game

    # Salvar o DataFrame atualizado como um arquivo CSV
    df_updated_games.to_csv(csv_file_path, index=False)

    # Converter o arquivo CSV em JSON
    csv_to_json(csv_file_path, file_path)

In [24]:
update_game('FM/B', "fd7f4c49-8cac-40f2-b696-fdd52a4f664f", "05", "01")

{'Comunicação Mackenzie': {'Medicina USP': 'Medicina USP', 'Comunicação Anhembi': 'E', 'Engenharia Mackenzie': 'E', 'Medicina ABC': 'E', 'Politécnica USP': 'E'}, 'Medicina USP': {'Comunicação Mackenzie': 'Medicina USP', 'Engenharia Mackenzie': 'E', 'Medicina ABC': 'E', 'Politécnica USP': 'E', 'Comunicação Anhembi': 'E'}, 'UNIFESP Osasco': {'Engenharia Anhembi': 'UNIFESP Osasco', 'Medicina Taubaté': 'E'}, 'Engenharia Anhembi': {'UNIFESP Osasco': 'UNIFESP Osasco', 'Economia Mackenzie': 'E'}, 'EACH USP': {'Economia Mackenzie': 'Economia Mackenzie', 'FMU': 'E'}, 'Economia Mackenzie': {'EACH USP': 'Economia Mackenzie', 'Engenharia Anhembi': 'E'}, 'Politécnica USP': {'Engenharia Mackenzie': 'Politécnica USP', 'Medicina ABC': 'E', 'Comunicação Anhembi': 'E', 'Medicina USP': 'E', 'Comunicação Mackenzie': 'E'}, 'Engenharia Mackenzie': {'Politécnica USP': 'Politécnica USP', 'Medicina USP': 'E', 'Comunicação Mackenzie': 'E', 'Comunicação Anhembi': 'E', 'Medicina ABC': 'E'}, 'Comunicação Anhembi':

In [ ]:
create_game(
    modality,
    'FFLCH USP',
    'Medicina Santo Amaro',
    '',
    '',
    '',
    'B',
    '',
    '',
    simulator=True)

In [45]:
modality = 'FM/D'
create_zero_game(
    modality,
    'FFLCH USP',
    'Medicina Santo Amaro',
    'B',
)

criado JSON em files/FM/D/games.json


In [7]:
modality = 'FM/D'
create_zero_game(
    modality,
    'CAAP UFABC',
    'IME USP',
    'B',
)
create_zero_game(
    modality,
    'Direito FMU',
    'IBMEC SP',
    'B',
)
create_zero_game(
    modality,
    'FFLCH USP',
    'Direito FMU',
    'B',
)
create_zero_game(
    modality,
    'IBMEC SP',
    'CAAP UFABC',
    'B',
)
create_zero_game(
    modality,
    'IME USP',
    'Medicina Santo Amaro',
    'B',
)
create_zero_game(
    modality,
    'Medicina UNIMES',
    'Medicina Anhembi',
    'A',
)
create_zero_game(
    modality,
    'Direito FMU',
    'Medicina Santo Amaro',
    'B',
)
create_zero_game(
    modality,
    'IBMEC SP',
    'IME USP',
    'B',
)

criado JSON em files/FM/D/games.json
criado JSON em files/FM/D/games.json
criado JSON em files/FM/D/games.json
criado JSON em files/FM/D/games.json
criado JSON em files/FM/D/games.json
criado JSON em files/FM/D/games.json
criado JSON em files/FM/D/games.json
criado JSON em files/FM/D/games.json


In [2]:
import os

def criar_diretorios(lista_strings, diretorio_base):
    os.makedirs('logs')
    for caminho in lista_strings:
        partes = caminho.split('/')
        caminho_atual = diretorio_base
        for parte in partes:
            caminho_atual = os.path.join(caminho_atual, parte)
            if not os.path.exists(caminho_atual):
                os.makedirs(caminho_atual)
                print(f'Criado diretório: {caminho_atual}')
        # Cria o diretório "group" dentro do último diretório criado
        caminho_group = os.path.join(caminho_atual, 'group')
        if not os.path.exists(caminho_group):
            os.makedirs(caminho_group)
            print(f'Criado diretório: {caminho_group}')

# Lista de strings
lista_string = [
    "FF/A",
    "FF/B",
    "FF/C",
    "FF/D",
    "FF/E",
    "FM/A",
    "FM/B",
    "FM/C",
    "FM/D",
    "FM/E",
    "FM/F",
 ]

# Diretório base
diretorio_base = 'files'

# Chamada da função
criar_diretorios(lista_string, diretorio_base)


Criado diretório: files\FF
Criado diretório: files\FF\A
Criado diretório: files\FF\A\group
Criado diretório: files\FF\B
Criado diretório: files\FF\B\group
Criado diretório: files\FF\C
Criado diretório: files\FF\C\group
Criado diretório: files\FF\D
Criado diretório: files\FF\D\group
Criado diretório: files\FF\E
Criado diretório: files\FF\E\group
Criado diretório: files\FM\C
Criado diretório: files\FM\C\group
Criado diretório: files\FM\E
Criado diretório: files\FM\E\group
Criado diretório: files\FM\F
Criado diretório: files\FM\F\group
Criado diretório: files\FM\D
Criado diretório: files\FM\D\group
